In [1]:
# !pip install -q pyLDAvis

In [2]:
# pip install BeautifulSoup4

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import operator
from nltk.corpus import stopwords
import gensim
from gensim import corpora, models
# from textblob import TextBlob
from bs4 import BeautifulSoup
import pyLDAvis.gensim_models 
import pickle 
import pyLDAvis
import tqdm

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('omw-1.4')

In [3]:
from nltk.corpus import stopwords
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

2024-04-17 11:22:08.627804: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-17 11:22:09.004051: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 11:22:09.004141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 11:22:09.084412: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-17 11:22:09.231672: I tensorflow/core/platform/cpu_feature_guar

{'become', 'was', 'whatever', 're', 'eight', 'serious', 'sixty', 'such', 'former', 'wherein', 'did', 'rather', 'two', 'above', 'nothing', 'already', 'since', 'quite', 'call', 'am', 'via', 'anything', 'beside', 'empty', 'beyond', 'these', 'should', 'always', 'twenty', 'too', 'whereas', 'else', 'somehow', 'third', "'re", 'front', 'what', 'few', 'whenever', 'fifteen', 'they', 'ca', 'before', 'at', 'because', 'done', 'all', 'thence', 'various', '‘s', 'much', 'noone', 'my', 'next', '‘d', 'wherever', 'formerly', 'anyone', 'onto', 'take', 'across', 'for', 'only', 'seem', 'yourselves', 'moreover', 'throughout', 'back', '’d', 'i', 'now', 'us', 'yours', 'thereafter', '’ve', 'were', 'make', 'twelve', 'be', 'using', 'themselves', 'therein', 'five', 'please', 'put', 'up', 'hereafter', 'sometimes', 'the', 'whose', 'elsewhere', 'why', 'own', 'hundred', 'fifty', 'therefore', 'once', 'had', 'cannot', 'there', 'until', 'yet', '’ll', 'not', 'into', 'could', 'even', "'d", 'nevertheless', 'when', 'show', '

In [4]:
def preprocess(wordlist):
  text=' '.join(wordlist)
  tokens = nltk.word_tokenize(text)
  l_nouns = [lemmatizer.lemmatize(word) for word in tokens]
  l_nouns = [word.lower() for word in l_nouns]
  nouns = ' '.join(l_nouns)
  doc = nlp(nouns)
  nouns = [doc[i] for i in range(len(doc)) if doc[i].tag_ == 'NNP' or doc[i].tag_ == 'NN' or doc[i].tag_ == 'NNPS' or doc[i].tag_ == 'NNS']
  nouns = [str(noun) for noun in nouns]
  return nouns

In [5]:
preprocess(["Today","Indian","India","joel","Thanks","Regards","IIT","Palakkad","anoop","George"])

['today',
 'indian',
 'india',
 'joel',
 'thanks',
 'iit',
 'palakkad',
 'anoop',
 'george']

In [6]:

# filename = "./Starred.mbox"

In [7]:
import mailbox
import re

In [10]:
# mbox_file_path = './Starred.mbox'
# mbox = mailbox.mbox(mbox_file_path)


In [8]:
def clean_and_filter(message):
    # Remove unwanted characters and formatting
    cleaned_message = re.sub(r'\n+', '\n', message)  # Remove extra line breaks
    cleaned_message = re.sub(r'<.*?>', '', cleaned_message)  # Remove HTML tags
    cleaned_message = re.sub(r'&nbsp;', ' ', cleaned_message)  # Replace &nbsp; with space
    cleaned_message = re.sub(r'.*Lost packet!.*\n', '', cleaned_message)
    cleaned_message = re.sub(r'^>.*$', '', cleaned_message, flags=re.MULTILINE)
    cleaned_message = re.sub(r'\n+', '\n', cleaned_message)  # Remove extra line breaks

    return cleaned_message


In [12]:
# output_file_path = 'fewMail.txt'
# yt = 0
# with open(output_file_path, 'w', encoding='utf-8') as output_file:
#     for message in mbox:
#         if message.is_multipart():
#             # For multipart messages, iterate through parts
#             for part in message.walk():
#                 if part.get_content_type() == "text/plain":
#                     text_content = part.get_payload(decode=True).decode('utf-8', 'ignore')
#                     cleaned_text = clean_and_filter(text_content)
#                     yt += 1
#                     output_file.write(cleaned_text + '\n' + "---------------------------------------------------------------" + '\n\n')
                    

#         else:
#             text_content = message.get_payload(decode=True).decode('utf-8', 'ignore')
#             cleaned_text = clean_and_filter(text_content)
#             output_file.write(cleaned_text + '\n')



In [9]:
doc = {}
doc_words = [[]]
termdf = {}
msg_doc = {}
no_of_docs = 0
msg_line_count = 0
output1_file = 'output_1000.txt'


In [14]:
for line in open(output1_file, encoding="utf-8"):
        if msg_line_count == 58000:
            break
        line = line.strip()
        if(line == "----------------------------------------------------"):
            doc_words[-1] = preprocess(doc_words[-1])
            doc_words.append([])
            msg_line_count = 0
            no_of_docs += 1
            for term in msg_doc:
                if (doc.__contains__(term)):
                    doc[term] = int(doc.get(term)) + msg_doc[term]
                else:
                    doc[term] = 1
            msg_doc.clear()
        else:
            msg_line_count+=1
            split = line.split(' ')
            for entry in split:
                # entry = entry.lower()
                if(entry in stopwords.words('english') or entry in sw_spacy or (not entry.isalnum()) or len(entry)<=2):
                    continue
                doc_words[-1].append(entry)
                if (msg_doc.__contains__(entry)):
                    msg_doc[entry] = int(msg_doc.get(entry)) + 1
                else:
                    msg_doc[entry] = 1
                    if(termdf.__contains__(entry)):
                        termdf[entry] = int(termdf.get(entry)) + 1
                    else:
                        termdf[entry] = 1

In [15]:
tfidf = {}
for term in doc:
    tfidf[term] = doc[term] * np.log(no_of_docs/termdf[term])

In [16]:
print("num docs", no_of_docs)

num docs 479


In [17]:
dictionary = corpora.Dictionary(doc_words)
corpus = [dictionary.doc2bow(text) for text in doc_words]

In [18]:
num_topics = 11
passes = 20
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics = num_topics, id2word = dictionary, passes = passes)

In [19]:
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(LDAvis_prepared)

In [20]:
from gensim import  models
from gensim.test.utils import datapath

In [21]:
lda_model.save("lda_model_11/lda_model_1")

In [22]:
# temp_file = datapath("./lda_model_11/lda_model_1")

In [23]:
lda_model = models.ldamodel.LdaModel.load("lda_model_11/lda_model_1")

In [24]:
tops = lda_model.print_topics(num_words = 50)
tops

(0,
  '0.038*"student" + 0.032*"scholarship" + 0.029*"institute" + 0.029*"fee" + 0.023*"thanks" + 0.022*"iit" + 0.022*"payment" + 0.020*"date" + 0.017*"palakkad" + 0.016*"application" + 0.012*"certificate" + 0.012*"link" + 0.010*"students" + 0.010*"contact" + 0.010*"form" + 0.010*"email" + 0.010*"mail" + 0.009*"tech" + 0.009*"information" + 0.008*"hostel" + 0.008*"year" + 0.008*"dean" + 0.007*"google" + 0.007*"bank" + 0.007*"end" + 0.007*"sector" + 0.006*"document" + 0.006*"number" + 0.006*"quarantine" + 0.006*"renewal" + 0.006*"request" + 0.005*"nsp" + 0.005*"income" + 0.005*"academics" + 0.005*"deadline" + 0.005*"inform" + 0.004*"semester" + 0.004*"college" + 0.004*"case" + 0.004*"start" + 0.004*"scheme" + 0.004*"reference" + 0.004*"nov" + 0.004*"roll" + 0.004*"detail" + 0.004*"engineering" + 0.004*"science" + 0.004*"note" + 0.004*"program" + 0.004*"computer"'),


[(0,
  '0.038*"student" + 0.032*"scholarship" + 0.029*"institute" + 0.029*"fee" + 0.023*"thanks" + 0.022*"iit" + 0.022*"payment" + 0.020*"date" + 0.017*"palakkad" + 0.016*"application" + 0.012*"certificate" + 0.012*"link" + 0.010*"students" + 0.010*"contact" + 0.010*"form" + 0.010*"email" + 0.010*"mail" + 0.009*"tech" + 0.009*"information" + 0.008*"hostel" + 0.008*"year" + 0.008*"dean" + 0.007*"google" + 0.007*"bank" + 0.007*"end" + 0.007*"sector" + 0.006*"document" + 0.006*"number" + 0.006*"quarantine" + 0.006*"renewal" + 0.006*"request" + 0.005*"nsp" + 0.005*"income" + 0.005*"academics" + 0.005*"deadline" + 0.005*"inform" + 0.004*"semester" + 0.004*"college" + 0.004*"case" + 0.004*"start" + 0.004*"scheme" + 0.004*"reference" + 0.004*"nov" + 0.004*"roll" + 0.004*"detail" + 0.004*"engineering" + 0.004*"science" + 0.004*"note" + 0.004*"program" + 0.004*"computer"'),
 (1,
  '0.024*"event" + 0.022*"course" + 0.022*"institute" + 0.020*"student" + 0.016*"iit" + 0.016*"palakkad" + 0.015*"alu

In [25]:
def parse_topic_model_input(input_list):
    output_list = {}
    all_words = {}
    for item in input_list:
        index, word_weights = item
        output_list[index] = {}
        words = [word.split("*")[1].strip().strip('"') for word in word_weights.split(" + ")]
        probs = [word.split("*")[0].strip().strip('"') for word in word_weights.split(" + ")]
        for i in range(len(words)):
          output_list[index][words[i]] = float(probs[i])
          if words[i] not in all_words:
            all_words[words[i]] = 1
          else: 
            all_words[words[i]] += 1
    return output_list,all_words

In [26]:
Tags_new,all_words = parse_topic_model_input(tops)


In [27]:
Tags_new


{0: {'student': 0.038,
  'scholarship': 0.032,
  'institute': 0.029,
  'fee': 0.029,
  'thanks': 0.023,
  'iit': 0.022,
  'payment': 0.022,
  'date': 0.02,
  'palakkad': 0.017,
  'application': 0.016,
  'certificate': 0.012,
  'link': 0.012,
  'students': 0.01,
  'contact': 0.01,
  'form': 0.01,
  'email': 0.01,
  'mail': 0.01,
  'tech': 0.009,
  'information': 0.009,
  'hostel': 0.008,
  'year': 0.008,
  'dean': 0.008,
  'google': 0.007,
  'bank': 0.007,
  'end': 0.007,
  'sector': 0.007,
  'document': 0.006,
  'number': 0.006,
  'quarantine': 0.006,
  'renewal': 0.006,
  'request': 0.006,
  'nsp': 0.005,
  'income': 0.005,
  'academics': 0.005,
  'deadline': 0.005,
  'inform': 0.005,
  'semester': 0.004,
  'college': 0.004,
  'case': 0.004,
  'start': 0.004,
  'scheme': 0.004,
  'reference': 0.004,
  'nov': 0.004,
  'roll': 0.004,
  'detail': 0.004,
  'engineering': 0.004,
  'science': 0.004,
  'note': 0.004,
  'program': 0.004,
  'computer': 0.004},
 1: {'event': 0.024,
  'course': 

In [28]:
def probability_normalizer():
  for topic in Tags_new:
    for word in Tags_new[topic]:
      if all_words[word] > 1:
        #print(Tags_new[topic][word])
        Tags_new[topic][word] = Tags_new[topic][word]/all_words[word]
    Tags_new[topic] = dict(sorted(Tags_new[topic].items(), key=lambda x:x[1],reverse=True))

In [29]:
probability_normalizer()

In [30]:
Tags_new


{0: {'date': 0.02,
  'scholarship': 0.016,
  'fee': 0.0145,
  'payment': 0.011,
  'tech': 0.009,
  'application': 0.008,
  'end': 0.007,
  'sector': 0.007,
  'certificate': 0.006,
  'quarantine': 0.006,
  'income': 0.005,
  'academics': 0.005,
  'student': 0.004222222222222222,
  'hostel': 0.004,
  'dean': 0.004,
  'college': 0.004,
  'start': 0.004,
  'scheme': 0.004,
  'reference': 0.004,
  'thanks': 0.003833333333333333,
  'bank': 0.0035,
  'students': 0.0033333333333333335,
  'mail': 0.0033333333333333335,
  'institute': 0.0032222222222222222,
  'number': 0.003,
  'renewal': 0.003,
  'iit': 0.00275,
  'year': 0.0026666666666666666,
  'contact': 0.0025,
  'nsp': 0.0025,
  'inform': 0.0025,
  'document': 0.002,
  'request': 0.002,
  'semester': 0.002,
  'nov': 0.002,
  'roll': 0.002,
  'engineering': 0.002,
  'science': 0.002,
  'computer': 0.002,
  'palakkad': 0.001888888888888889,
  'information': 0.0018,
  'link': 0.0015,
  'email': 0.0014285714285714286,
  'case': 0.0013333333333

In [31]:
# nltk.download('stopwords')
# nltk.download('punkt')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams

def extract_keywords(text):
    stop_words = set(stopwords.words("english"))
    words = nltk.word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha()]
    words = [word for word in words if word not in stop_words]
    word_ngrams = ngrams(words, 2)
    word_ngrams = [' '.join(ngram) for ngram in word_ngrams]
    words += word_ngrams
    word_freq = nltk.FreqDist(words)
    keywords = word_freq.most_common(10)
    keywords = [keyword for keyword, freq in keywords]
    return keywords

def cluster_name(text):
    keywords = extract_keywords(text)
    return keywords

In [34]:
def Tag_prediction(filename):
  with open(filename,'r') as f:
    text = f.read()
  new_text_corpus =  dictionary.doc2bow(text.split())
  prob_topics = lda_model.get_document_topics(new_text_corpus)
  prob_topics.sort(key = lambda x: x[1],reverse=True)
  print(prob_topics)
  output_tags = []
  cluster_sugg = cluster_name(text)
  print(cluster_sugg)
  terms = text.split()
  output_tags = list(Tags_new[prob_topics[0][0]].keys())[:4]
  print(output_tags)
  return output_tags

In [35]:
Tag_prediction("./test1.txt")

[(2, 0.4390903), (5, 0.13995585), (0, 0.09822309), (1, 0.08918508), (8, 0.0815695), (4, 0.07112984), (10, 0.058308918), (6, 0.018521683)]
['medical', 'officer', 'visit', 'hospital', 'medical officer', 'medicine', 'consultation', 'phone', 'emergency', 'department']
['day', 'july', 'science', 'katragadda']


['day', 'july', 'science', 'katragadda']